In [1]:
import os
import sys
import pandas as pd
import ast
import numpy as np
os.chdir('../')


In [3]:
meta_data_path = "../data/metadata_subset.csv"
reviews_data_path = "../data/reviews_subset.csv"
meta_data = pd.read_csv(meta_data_path)
review_data = pd.read_csv(reviews_data_path)
meta_data["asin"] = meta_data["asin"].astype(str)
review_data["asin"] = review_data["asin"].astype(str)

In [4]:
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"['Software', 'Education &amp; Reference']",NaN,"['Slides with Video, Teaching Public Speaking ...",NaN,Instructor's Resource CD-ROM for The Art of Sp...,[],NaN,McGraw Hill,[],"18,178 in Software (",[],Software,NaN,</div>,$8.00,007742817X,NaN
1,"['Software', 'Education &amp; Reference']",NaN,"[""Contains a guided tour of the program, Plann...",NaN,Magruder's American Government Resource Pro CD...,[],NaN,Magruder's,[],"19,702 in Software (",['0130679550'],Software,NaN,</div>,NaN,0130438480,NaN
2,"['Software', 'Education & Reference', 'Test Pr...",NaN,[],NaN,Prentice Hall Test Manager a Comprehensive Sui...,[],NaN,prentice hall,[],"54,036 in Software (",[],Software,NaN,</div>,NaN,0130852414,NaN
3,"['Software', 'Education &amp; Reference', 'Tes...",NaN,"['Windos 95, 98, NT4, 200, XP\nMac OS 9.1-9.2 ...",NaN,Magruder's American Government Itext Interacti...,[],NaN,Magruder's,['Interactive Learning Tools-Bring Content to ...,"52,031 in Software (",[],Software,NaN,</div>,NaN,0131817949,NaN
4,"['Software', 'Design &amp; Illustration', 'CAD']",NaN,"['2.5 Floppy', '', '']",NaN,AUTOCAD The Student Edition Release 10 (1982-89),[],NaN,Autodesk,[],"30,901 in Software (",[],Software,NaN,</div>,NaN,0201656302,NaN


In [4]:
review_data.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,77613252,Michelle W,The materials arrived early and were in excell...,Material Great,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,77613252,Rosalind White Ames,I am really enjoying this book with the worksh...,Health,NaN
2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,77613252,Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,7.0
3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,77613252,Lucy,This book was missing pages!!! Important pages...,missing pages!!,3.0
4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,77775473,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,NaN


### Cleaning meta_data

In [5]:
meta_data['price'] = meta_data.price.str.replace('$','')
meta_data['price'] = meta_data.price.str[:5]
meta_data['price'] = meta_data['price'].str.extract('([0-9]+.[0-9]*)')
meta_data['price'] = meta_data['price'].str.strip()
meta_data['price'] = meta_data['price'].str.replace(',','')
meta_data['price'] = meta_data['price'].astype(float)
meta_data = meta_data[meta_data.price.notnull()]
meta_data['software_category'] = meta_data['category'].apply(lambda x: ast.literal_eval(x)[1])

In [5]:
meta_data.columns.tolist()

['category',
 'tech1',
 'description',
 'fit',
 'title',
 'also_buy',
 'tech2',
 'brand',
 'feature',
 'rank',
 'also_view',
 'main_cat',
 'similar_item',
 'date',
 'price',
 'asin',
 'details',
 'software_category']

### Generating other features

In [6]:
Min_license_fee = meta_data.groupby('software_category').agg(Min_Licensing_Fee=('price', np.min)).reset_index()
meta_data = meta_data.merge(Min_license_fee, on='software_category', how='inner')
meta_data['Licensing_Fee'] = meta_data['Min_Licensing_Fee'] * 0.8
meta_data.drop(columns=['Min_Licensing_Fee'],inplace=True)
meta_data['Implemention_cost'] = meta_data.apply(lambda row: row['price']*0.5, axis=1)
meta_data['Maintenance_cost'] = meta_data.apply(lambda row: row['price']*0.1, axis=1)

In [7]:
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,main_cat,similar_item,date,price,asin,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost
0,"['Software', 'Education &amp; Reference']",NaN,"['Slides with Video, Teaching Public Speaking ...",NaN,Instructor's Resource CD-ROM for The Art of Sp...,[],NaN,McGraw Hill,[],"18,178 in Software (",...,Software,NaN,</div>,8.00,007742817X,NaN,Education &amp; Reference,5.592,4.000,0.800
1,"['Software', 'Education &amp; Reference']",NaN,"[""Over 50 Activities Developing Essential 1st ...",NaN,SchoolHouse Rock! 1st &amp; 2nd Grade [Win Mac...,[],NaN,Scholastic,['SchoolHouse Rock! 1st &amp; 2nd Grade [Win M...,"12,202 in Software (",...,Software,NaN,</div>,31.47,043983595X,NaN,Education &amp; Reference,5.592,15.735,3.147
2,"['Software', 'Education &amp; Reference', 'Lan...",NaN,['Learn Japanese with the long-awaited sequel ...,NaN,Human Japanese Intermediate,['B00NQI1XAE'],NaN,Brak Software,"['More than 40 illuminating chapters, presente...","6,605 in Software (",...,Software,NaN,</div>,9.98,061591067X,NaN,Education &amp; Reference,5.592,4.990,0.998
3,"['Software', 'Education &amp; Reference']",NaN,['Makes teching and learning mathematics easie...,NaN,Scientific Notebook Version 4.0,[],NaN,MacKichan,['Mathematical word processing plus the easies...,"60,352 in Software (",...,Software,NaN,</div>,74.50,0971562431,NaN,Education &amp; Reference,5.592,37.250,7.450
4,"['Software', 'Education &amp; Reference', 'Lan...",NaN,['The ASL Tales and Games for Kids CD series f...,NaN,ASL Tales and Games for Kids - Leash Lane,[],NaN,IDRT,['The ASL Tales and Games for Kids CD series f...,"14,297 in Software (",...,Software,NaN,</div>,18.00,0981148603,NaN,Education &amp; Reference,5.592,9.000,1.800


### Understanding Preprocessed Data

In [2]:
import seaborn as sns

In [3]:
review_meta = pd.read_csv("../data/review_metadata.csv")

/tmp/ipykernel_155287/710226407.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  review_meta = pd.read_csv("../data/review_metadata.csv")


In [5]:
review_meta.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote,...,also_view,main_cat,similar_item,date,price,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost
0,0,5.0,False,"07 23, 2008",A8IOST6U6WH9B,0615179088,C. Radey,Human Japanese is a truly superb introduction ...,Human Japanese,12,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
1,1,5.0,False,"06 4, 2008",A1MUV9F35OROS5,0615179088,D. Abel,I got Human Japanese as a demo from its websit...,Best Japanese Program Available,11,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
2,2,4.0,False,"04 8, 2008",A27PAMABWVQ892,0615179088,piepiepie75,My first experience with Human Japanese was th...,Better than the Human Japanese 1...but not muc...,99,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
3,3,5.0,False,"03 26, 2008",A3HWWVK0L3JEKF,0615179088,K. Grier,This is the first language software that I hav...,Great Product,4,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
4,4,5.0,False,"02 20, 2008",A3NO2V2JU4Y8UY,0615179088,H. Granat,Human japanese is the best pc program for lear...,Love it!,2,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994


In [6]:
review_meta.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'vote', 'category', 'tech1',
       'description', 'fit', 'title', 'also_buy', 'tech2', 'brand', 'feature',
       'rank', 'also_view', 'main_cat', 'similar_item', 'date', 'price',
       'details', 'software_category', 'Licensing_Fee', 'Implemention_cost',
       'Maintenance_cost'],
      dtype='object')

In [7]:
review_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145863 entries, 0 to 145862
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         145863 non-null  int64  
 1   overall            145863 non-null  float64
 2   verified           145863 non-null  bool   
 3   reviewTime         145863 non-null  object 
 4   reviewerID         145863 non-null  object 
 5   asin               145863 non-null  object 
 6   reviewerName       145836 non-null  object 
 7   reviewText         145834 non-null  object 
 8   summary            145844 non-null  object 
 9   vote               30493 non-null   object 
 10  category           145863 non-null  object 
 11  tech1              26 non-null      object 
 12  description        145863 non-null  object 
 13  fit                0 non-null       float64
 14  title              145863 non-null  object 
 15  also_buy           145863 non-null  object 
 16  te

In [8]:
review_meta.describe()

,Unnamed: 0,overall,fit,tech2,similar_item,price,Licensing_Fee,Implemention_cost,Maintenance_cost
count,145863.000000,145863.000000,0.0,0.0,0.0,145863.000000,145863.000000,145863.000000,145863.000000
mean,72931.000000,3.749224,NaN,NaN,NaN,61.873790,1.023924,30.936895,6.187379
std,42107.165495,1.584138,NaN,NaN,NaN,82.919461,1.569105,41.459731,8.291946
min,0.000000,1.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,36465.500000,2.000000,NaN,NaN,NaN,14.990000,0.000000,7.495000,1.499000
50%,72931.000000,5.000000,NaN,NaN,NaN,36.880000,0.008000,18.440000,3.688000
75%,109396.500000,5.000000,NaN,NaN,NaN,79.990000,1.272000,39.995000,7.999000
max,145862.000000,5.000000,NaN,NaN,NaN,3175.000000,11.960000,1587.500000,317.500000


In [ ]:
review_meta